In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from credentials import user
from credentials import server
from credentials import password
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
businesses_df = pd.read_csv("../data/businesses.csv",encoding='latin-1')
clusters_df = pd.read_csv("../data/clusters.csv",encoding='latin-1')
violations = pd.read_csv("../data/hubNashville__311__Service_Requests.csv")
violations=violations[violations['Request Type']=='COVID-19'].reset_index(drop=True)
violations=violations[violations['Subrequest Type']=='COVID-19 Violations'].reset_index(drop=True)

C:\Users\lapto\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
businesses_df

,address,vicinity,name,type
0,"2440 COUCHVILLE PIKE, Nashville","2440 Couchville Pike, Nashville",Wilson Sporting Goods Co,"['point_of_interest', 'establishment']"
1,"1613 Buchanan St, Nashville","1613 Buchanan Street, Nashville",Coin Cloud Bitcoin ATM,"['atm', 'finance', 'point_of_interest', 'estab..."
2,"1613 Buchanan St, Nashville","1613 Buchanan Street, Nashville",Discounts Mart,"['convenience_store', 'food', 'point_of_intere..."
3,"1613 Buchanan St, Nashville","1613 Buchanan Street, Nashville",Northwest Liquors,"['liquor_store', 'point_of_interest', 'store',..."
4,"306 White Bridge Pike, Nashville","306 White Bridge Pike, Nashville",Suds Up Car Wash of West Nashville,"['car_wash', 'point_of_interest', 'establishme..."
...,...,...,...,...
7261,"131 W Express Dr, Nashville","131 West Express Drive, Nashville",Intermodal Cartage Group (IMCG),"['moving_company', 'point_of_interest', 'estab..."
7262,"2400 Gallatin Pike N, Madison","2400 Gallatin Pike North, Madison",Nashville Toyota North,"['finance', 'car_dealer', 'point_of_interest',..."
7263,"730 2nd Ave S, Nashville","730 2nd Avenue South, Nashville",Davidson County Sports Authority,"['local_government_office', 'point_of_interest..."
7264,"105 Broadway, Nashville","105 Broadway, Nashville",Nashville Underground,"['night_club', 'bar', 'restaurant', 'food', 'p..."


In [4]:
clusters_df

,Cluster Name,Facility Type,Cluster Start Date,# Cases,Latitude,Longitude
0,Vanderbilt Parties,Social Gathering,3/11/2020,49,36.144703,-86.802655
1,Event at Clementine Hall,Social Gathering,3/14/2020,23,36.152444,-86.844583
2,Religious Retreat,Social Gathering,3/25/2020,18,NaN,NaN
3,The Health Center at Richland Place,LTCF,4/3/2020,47,36.128750,-86.817345
4,Trevecca Center for Rehab and Healing,LTCF,4/4/2020,102,36.144562,-86.754560
...,...,...,...,...,...,...
57,One Stone Church Service,Church,9/22/2020,10,36.189960,-86.765149
58,Miss Kelli's,Bar,10/1/2020,14,36.164369,-86.778515
59,Nashville Rescue Mission - Women's Campus,Congregate Living,10/1/2020,70,36.153700,-86.777276
60,Link Systems Electric,Other,10/16/2020,12,36.092924,-86.741684


In [7]:
violations

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,444823,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,08/14/2020 02:38:00 AM,08/14/2020 02:37:15 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,2045 Antioch Pike,ANTIOCH,28.0,37013,36.068546,-86.683898,"(36.0685457, -86.6838975)"
1,479965,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/13/2020 11:48:40 AM,10/13/2020 01:48:08 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,2800 Opryland Dr,NASHVILLE,15.0,37214,36.211269,-86.694302,"(36.211269429469425, -86.6943024067459)"
2,392927,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,06/16/2020 12:35:20 AM,06/17/2020 02:28:19 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,2607 12th Ave S,NASHVILLE,18.0,37204,36.123500,-86.790130,"(36.1235, -86.79013)"
3,444512,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,08/13/2020 06:50:31 PM,08/14/2020 02:22:11 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,3200 Clarksville Pike,NASHVILLE,2.0,37218,36.194163,-86.831394,"(36.1941628, -86.8313936)"
4,444440,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,08/13/2020 06:03:09 PM,08/14/2020 02:18:59 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,20 Grundy St,NASHVILLE,19.0,37203,36.158440,-86.788038,"(36.15843971160494, -86.7880380629013)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11883,500977,New,COVID-19,COVID-19 Violations,COVID-19 Violations,11/12/2020 04:36:53 PM,NaN,Phone,NaN,NaN,...,False,NaN,False,410 Fieldcrest Dr,NASHVILLE,26.0,37211,36.072223,-86.756820,"(36.0722229, -86.7568201)"
11884,500968,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,11/12/2020 04:31:32 PM,11/13/2020 03:29:22 PM,hubNashville Community,NaN,NaN,...,False,NaN,False,2037 Jefferson St,NASHVILLE,21.0,37208,36.168645,-86.811969,"(36.168645211609274, -86.8119685932541)"
11885,501064,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,11/12/2020 05:15:42 PM,11/13/2020 03:29:58 PM,Phone,NaN,NaN,...,False,NaN,False,705 Sawyer Brown Rd,NASHVILLE,22.0,37221,36.068370,-86.949250,"(36.06837, -86.94925)"
11886,501127,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,11/12/2020 05:41:13 PM,11/13/2020 03:37:11 PM,Phone,NaN,NaN,...,False,NaN,False,3805 Park Ave,NASHVILLE,24.0,37209,36.151270,-86.831430,"(36.15127, -86.83143)"


In [5]:
clusters_df.columns = ['name','facility','start_date','cases','lat','lng']
clusters_df.head()

,name,facility,start_date,cases,lat,lng
0,Vanderbilt Parties,Social Gathering,3/11/2020,49,36.144703,-86.802655
1,Event at Clementine Hall,Social Gathering,3/14/2020,23,36.152444,-86.844583
2,Religious Retreat,Social Gathering,3/25/2020,18,NaN,NaN
3,The Health Center at Richland Place,LTCF,4/3/2020,47,36.128750,-86.817345
4,Trevecca Center for Rehab and Healing,LTCF,4/4/2020,102,36.144562,-86.754560


In [8]:
clusters_add = pd.merge(clusters_df, businesses_df, on='name')
clusters_add

,name,facility,start_date,cases,lat,lng,address,vicinity,type
0,Cargill,Commercial-Warehouse,4/21/2020,22,36.114254,-86.754589,"2621 Eugenia Ave, Nashville","2621 Eugenia Avenue, Nashville","['food', 'point_of_interest', 'establishment']"
1,Cargill,Commercial-Warehouse,4/21/2020,22,36.114254,-86.754589,"2621 Eugenia Ave, Nan","2621 Eugenia Avenue, Nashville","['food', 'point_of_interest', 'establishment']"
2,The Opal at Music City,LTCF,6/3/2020,27,36.161851,-86.661030,"2733 McCampbell Ave, Nashville","2733 McCampbell Avenue, Nashville","['health', 'point_of_interest', 'establishment']"
3,Hermitage Hall,Congregate Living,6/20/2020,93,36.140095,-86.777566,"1220 8th Ave S, Nashville","1220 8th Avenue South, Nashville","['health', 'point_of_interest', 'establishment']"
4,Standing Tall Music City,Congregate Living,7/1/2020,50,36.201781,-86.862279,"3981 Stewarts Ln, Nashville","3981 Stewarts Lane, Nashville","['school', 'point_of_interest', 'establishment']"
5,Tennessee Titans,Other,9/12/2020,23,NaN,NaN,"460 Great Circle Rd, Nashville","460 Great Circle Road, Nashville","['point_of_interest', 'establishment']"
